In [1]:
import PyStrategy
import pandas as pd
import matplotlib.pyplot as plt

# Data fetching

In [2]:
import requests
import zipfile
import io

def download_binance_futures_data(url):
    """
    Downloads and extracts the CSV from the Binance zip file.
    """
    response = requests.get(url)
    if response.status_code == 200:
        bytes_io = io.BytesIO(response.content)
        with zipfile.ZipFile(bytes_io, 'r') as z:
            csv_filename = z.namelist()[0]
            print(f"Extracting file: {csv_filename}")
            with z.open(csv_filename) as csvfile:
                df = pd.read_csv(csvfile)
        return df
    else:
        raise Exception(f"Failed to download file. Status code: {response.status_code}")

def download_binance_spot_data(url):
    """
    Downloads and extracts the CSV from the Binance zip file.
    """
    response = requests.get(url)
    if response.status_code == 200:
        bytes_io = io.BytesIO(response.content)
        with zipfile.ZipFile(bytes_io, 'r') as z:
            # Assume there's one CSV file in the archive
            csv_filename = z.namelist()[0]
            print(f"Extracting file: {csv_filename}")
            with z.open(csv_filename) as csvfile:
                df = pd.read_csv(csvfile)
        df.columns = ["id", "price", "quantity", "start_id", "end_id", "transact_time", "is_buyer_maker", "flag"]
        return df
    else:
        raise Exception(f"Failed to download file. Status code: {response.status_code}")

def transform_binance_trades(df, symbol='BTCUSDT', isSpot = False):
    """
    Transforms the Binance aggTrades DataFrame into the format:
      - symbol: LowCardinality(String) [constant value]
      - price: Float32
      - quantity: Float32
      - side: Enum8('BUY' = 0, 'SELL' = 1) determined from is_buyer_maker
      - event_timestamp: DateTime64(9) converted from transact_time (ms)
    """
    
    transformed_df = pd.DataFrame()

    #transformed_df['id'] = df['agg_trade_id']
    # Set a constant symbol (can be parameterized)
    transformed_df['symbol'] = symbol
    
    # Convert price and quantity to Float32
    transformed_df['price'] = df['price'].astype('float32')
    transformed_df['quantity'] = df['quantity'].astype('float32')
    
    # Determine side: if is_buyer_maker is True, then the maker is the buyer,
    # meaning the aggressor was selling. Otherwise, it's a BUY.
    transformed_df['side'] = df['is_buyer_maker'].apply(lambda x: 'SELL' if x else 'BUY')

    transformed_df['event_timestamp'] = pd.to_datetime(df['transact_time'], unit='ms' if not isSpot else 'us').round('ms')
    transformed_df['symbol'] = symbol
    return transformed_df

if __name__ == "__main__":
    symbol = "THEUSDT"
    url = f"https://data.binance.vision/data/futures/um/daily/aggTrades/{symbol}/{symbol}-aggTrades-2025-02-14.zip"
    print(url)
    df = transform_binance_trades(download_binance_futures_data(url), symbol)
    print("Futures loaded")
    df['iside'] = df['side'].replace({'BUY':1, 'SELL':-1})*df.quantity
    df = pd.concat([df[['event_timestamp', 'price']].groupby('event_timestamp').mean(),
            df[['event_timestamp', 'quantity']].groupby('event_timestamp').sum(),
            df[['event_timestamp', 'iside']].groupby('event_timestamp').sum()], 
            axis = 1)
    df['side'] = (df['iside'] > 0).replace({True: PyStrategy.Side.Buy, False: PyStrategy.Side.Sell})
    df['symbol'] = symbol+'_future'
    future_trades = df
    url = f"https://data.binance.vision/data/spot/daily/aggTrades/{symbol}/{symbol}-aggTrades-2025-02-14.zip"
    print(url)
    spot_trades = transform_binance_trades(download_binance_spot_data(url), symbol, True)
    spot_trades['event_timestamp'] = spot_trades.event_timestamp.apply(lambda x: x.round('ms'))
    spot_trades['iside'] = spot_trades['side'].replace({'BUY':1, 'SELL':-1})*spot_trades.quantity
    spot_trades = pd.concat([spot_trades[['event_timestamp', 'price']].groupby('event_timestamp').mean(),
            spot_trades[['event_timestamp', 'quantity']].groupby('event_timestamp').sum(),
            spot_trades[['event_timestamp', 'iside']].groupby('event_timestamp').sum()], 
            axis = 1)
    spot_trades['side'] = (spot_trades['iside'] > 0).replace({True: PyStrategy.Side.Buy, False: PyStrategy.Side.Sell})
    spot_trades['symbol'] = symbol+'_spot'

https://data.binance.vision/data/futures/um/daily/aggTrades/THEUSDT/THEUSDT-aggTrades-2025-02-14.zip
Extracting file: THEUSDT-aggTrades-2025-02-14.csv
Futures loaded
https://data.binance.vision/data/spot/daily/aggTrades/THEUSDT/THEUSDT-aggTrades-2025-02-14.zip


/tmp/ipykernel_872103/4253031412.py:73: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['iside'] = df['side'].replace({'BUY':1, 'SELL':-1})*df.quantity


Extracting file: THEUSDT-aggTrades-2025-02-14.csv


/tmp/ipykernel_872103/4253031412.py:85: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  spot_trades['iside'] = spot_trades['side'].replace({'BUY':1, 'SELL':-1})*spot_trades.quantity


# Price visualization

In [ ]:
#prices
df['price'].plot()
spot_trades['price'].plot(alpha=0.3)

In [ ]:
#show spot-future spread
spread = pd.DataFrame({'future_price':future_trades['price'].resample('100ms').last().dropna(),
            'spot_price':spot_trades['price'].resample('100ms').last().dropna()}).dropna()

spread = spread.future_price - spread.spot_price
spread.plot()

# Data storage creation

In [ ]:
#create data storage that will contain all the price data that will be used by the backtest
dataStorage = PyStrategy.DataStorage() 

# Backtest

## Simple ladder strategy

In [ ]:
#adding data
dataStorage.AddVMDTrades(
    "THEUSDT_futures",
    df.index.astype(int), #nanosecond ts, any other resolution will work as well as long as order is the same for all time series provided
    df.price,
    df.quantity,
    df.side,
    df.symbol
)

In [ ]:
#create a strategy class that will act on trade events
class SimpleLadder(PyStrategy.Strategy):
    def __init__(self):
        super().__init__(0,0, dataStorage)
        self.PriceLevel = 0.8
        self.Size = 0.75
        self.Count = 10
        self.Delta = 0.03
        self.Activated = False
        self.ActivTS = 0
        self.Removed = False

    def sendQuotes(self):
        for i in range(self.Count):
            order = self.SendOrder("THEUSDT", 
                                         self.PriceLevel + i*self.Delta, 
                                         self.Size, 
                                         PyStrategy.Side.Sell, 
                                         PyStrategy.OrderType.Limit, 
                                         f"Random quote #{i}")
        
    def OnTrade(self, trade):
        if (trade.Price > 0.8 and not self.Activated):
            self.sendQuotes()
            self.Activated = True
            self.ActivTS = trade.LocalTimestamp
        elif (self.Activated and trade.LocalTimestamp - self.ActivTS > 10*1e9 and not self.Removed):
            self.Removed = True
            for order in self.orders:
                print(order.Text)
                if (order.State == PyStrategy.OrderState.Active):
                    self.CancelOrder(order)

    def OnNewOrder(self, order):
        print(f"New order: {order.to_string()}")

    def OnOrderFilled(self, order):
        print(f"Filled order: {order.to_string()}")

In [ ]:
strategy = SimpleLadder()
strategy.CommitData()

In [ ]:
strategy.Run()

In [ ]:
#show ALL orders
pd.DataFrame(strategy.GetOrders())

In [ ]:
#show FILLED orders
pd.DataFrame(strategy.GetFilledOrders())

## Spreads trading

In [ ]:
#adding data
dataStorage.AddVMDTrades(
    "THEUSDT_spot",
    spot_trades.index.astype(int), #nanosecond ts, any other resolution will work as well as long as order is the same for all time series provided
    spot_trades.price,
    spot_trades.quantity,
    spot_trades.side,
    spot_trades.symbol
)

In [ ]:
class SpreadQuoter(PyStrategy.Strategy):
    def __init__(self, executionLatencyMs, marketDataLatencyMs):
        super().__init__(int(executionLatencyMs*1e6), int(marketDataLatencyMs*1e6), dataStorage) #providing 150ms latency both on md and execution
        self.MaxLongPosition = 150
        self.MaxShortPosition = 150
        self.Sensitivity = 0.01
        self.Spread = 0
        self.BidOffset = 0.1
        self.AskOffset = 0.1
        self.OrderSize = 10

        self.__lastUpdatePrice = 0
        self.__position = 0 #single aggregated spread position
        self.__bidQuote = None
        self.__askQuote = None
        self.__spreadOrderId = 0

    def cancelIfLegit(self, order):
        if (order is not None and order.State == PyStrategy.OrderState.Active):
            self.CancelOrder(order)
    
    def TrySendBidQuote(self):
        self.cancelIfLegit(self.__bidQuote)
        
        if (self.__position >= self.MaxLongPosition):
            return False

        self.__bidQuote = self.SendOrder('THEUSDT_future', 
                        self.__lastUpdatePrice - self.BidOffset + self.Spread,
                        min(self.OrderSize, self.MaxLongPosition - self.OrderSize),
                        PyStrategy.Side.Buy,
                        PyStrategy.OrderType.Limit, 
                        f"{self.__spreadOrderId}b")
        return True     
        

    def TrySendAskQuote(self):
        self.cancelIfLegit(self.__askQuote)
        
        if (-self.__position >= self.MaxShortPosition):
            return False

        self.__askQuote = self.SendOrder('THEUSDT_future', 
                        self.__lastUpdatePrice + self.AskOffset + self.Spread,
                        min(self.OrderSize, self.MaxShortPosition - self.OrderSize),
                        PyStrategy.Side.Sell,
                        PyStrategy.OrderType.Limit, 
                        f"{self.__spreadOrderId}a")
        return True     
        
    def OnTrade(self, trade):
        if (trade.Instrument == 'THEUSDT_future'):
            return

        if (abs(self.__lastUpdatePrice - trade.Price) > self.Sensitivity):
            self.__lastUpdatePrice = trade.Price
            self.__spreadOrderId += 1
            self.TrySendBidQuote()
            self.TrySendAskQuote()

    def OnOrderFilled(self, order):
        if (order.Instrument == 'THEUSDT_spot'):
            return
            
        self.SendOrder('THEUSDT_spot', 
                        self.__lastUpdatePrice,
                        order.FilledQty,
                        PyStrategy.Side.Buy if order.OrderSide == PyStrategy.Side.Sell else PyStrategy.Side.Sell,
                        PyStrategy.OrderType.Market,
                        order.Text)

        self.__position += order.FilledQty if order.OrderSide == PyStrategy.Side.Buy else -order.FilledQty

In [ ]:
strategy = SpreadQuoter(30, 15)
strategy.CommitData()

In [ ]:
strategy.Run()

In [ ]:
print(f"It was sent {len(strategy.GetOrders())} orders, filled {len(strategy.GetFilledOrders())}")

In [ ]:
spread_trades = pd.DataFrame(strategy.GetFilledOrders()).set_index('text')
spread_trades = pd.DataFrame({'spread': spread_trades[spread_trades['instrument'] == 'THEUSDT_future']['exec_price'] - spread_trades[spread_trades['instrument'] == 'THEUSDT_spot']['exec_price'],
                 'side': spread_trades[spread_trades['instrument'] == 'THEUSDT_future']['side'],
                 'filled_qty': spread_trades[spread_trades['instrument'] == 'THEUSDT_future']['filled_qty'],
                 'ts_first_leg_executed': spread_trades[spread_trades['instrument'] == 'THEUSDT_future']['last_report_timestamp'],
                 'ts_second_leg_executed': spread_trades[spread_trades['instrument'] == 'THEUSDT_spot']['last_report_timestamp']})
spread_trades['ts_first_leg_executed'] = pd.to_datetime(spread_trades['ts_first_leg_executed'])
spread_trades['ts_second_leg_executed'] = pd.to_datetime(spread_trades['ts_second_leg_executed'])

In [ ]:
#show spread trades
spread_trades